<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/Find_tuning_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore

In [3]:
import pickle
import joblib

######Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

train 3 class

In [6]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

#### train 3 class  --- สร้างตารางเก็บ ชื่อภาพ และชื่อคลาส
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 451 ภาพ TB150, Normal151 และ CA150 (CA คือ lungcancer)

In [7]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [8]:
normal = []
for i in range(len(os.listdir(Normal151_dir))):
    img = os.listdir(Normal151_dir)[i]
    normal.append(img)
    
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [9]:
tuberculosis = []
for i in range(len(os.listdir(TB150_dir))):
    img = os.listdir(TB150_dir)[i]
    tuberculosis.append(img)

ttuberculosis = pd.DataFrame({'ImageName':[tuberculosis][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [10]:
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [11]:
data_train3 = pd.concat([tnormal, ttuberculosis, tlungcancer])
data_train3[-3:]

,ImageName,NameType,NumberType
147,ca26.jpg,Lungcancer,2
148,ca7.jpg,Lungcancer,2
149,ca8.jpg,Lungcancer,2


####  test 3 class จากเน็ต

In [ ]:
TB700_dir = os.path.join(data_dir,'test_model/Tuberculosis700')
Normal94_dir = os.path.join(data_dir,'test_model/NORMAL94')
CA150_dir = os.path.join(data_dir,'CA150')                  #อันเดิมที่ใช้เทรน

AllTest_2class = os.path.join(data_dir,'test_model/AllTest_2class')

In [ ]:
#สำหรับ test model
normal = []
for i in range(len(os.listdir(Normal94_dir))):
    img = os.listdir(Normal94_dir)[i]
    normal.append(img)
    
tnormal94 = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [ ]:
#สำหรับ test model
tb700 = []
for i in range(len(os.listdir(TB700_dir))):
    img = os.listdir(TB700_dir)[i]
    tb700.append(img)

ttb700 = pd.DataFrame({'ImageName':[tb700][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [ ]:
#สำหรับ test model ข้อม฿ลเดิม
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [ ]:
data_test2 = pd.concat([tnormal94[:30], ttb700[:30], tlungcancer[:10]])

In [ ]:
test_NorTB = []
for ImageName in tqdm(data_test2['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = AllTest_2class +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))
    test_NorTB.append(img)

100%|██████████| 70/70 [00:19<00:00,  3.60it/s]


In [ ]:
data_test = np.array(test_NorTB, dtype="float32") / 255.0

labels_test = np.array(data_test2[['NameType']] )        
labels_test.shape = (len(labels_test),) 

le3_Incep = LabelEncoder()                                                                     
labels_test = le3_Incep.fit_transform(labels_test)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels_test = to_categorical(labels_test, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

## K-fold  -- InceptionV3

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [01:32<00:00,  4.87it/s]


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

In [ ]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model3_Incep.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_Incep = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_Incep.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
kf3_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf3_Incep.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train3_Incep = data3_Incep[train]
    y_train3_Incep = labels3_Incep[train]
    x_test3_Incep = data3_Incep[test]
    y_test3_Incep = labels3_Incep[test]

    H3_Incep = model3_Incep.fit( x=aug.flow(x_train3_Incep, y_train3_Incep, batch_size=BS),
	                                            validation_data=(x_test3_Incep, y_test3_Incep),
	                                            steps_per_epoch=len(x_train3_Incep) // BS,
	                                            epochs=10,  callbacks = [callback])
    
    pred3_Incep = model3_Incep.predict(x_test3_Incep)
    
    oos_y.append(y_test3_Incep)
    oos_pred.append(pred3_Incep)  
    print(classification_report(y_test3_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred3_Incep.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(y_test3_Incep.argmax(axis=1),	        #ค่า acc
#                                               pred3_Incep.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(testY3_Incep.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_Incep.argmax(axis=1) ) )

Fold #1
Epoch 1/10
45/45 [==============================] - 103s 2s/step - loss: 0.6675 - accuracy: 0.3194 - val_loss: 0.6413 - val_accuracy: 0.3956
Epoch 2/10
45/45 [==============================] - 102s 2s/step - loss: 0.6375 - accuracy: 0.4111 - val_loss: 0.6231 - val_accuracy: 0.4835
Epoch 3/10
45/45 [==============================] - 97s 2s/step - loss: 0.6219 - accuracy: 0.4472 - val_loss: 0.6100 - val_accuracy: 0.5165
Epoch 4/10
45/45 [==============================] - 99s 2s/step - loss: 0.6082 - accuracy: 0.4806 - val_loss: 0.5993 - val_accuracy: 0.5495
Epoch 5/10
45/45 [==============================] - 98s 2s/step - loss: 0.5958 - accuracy: 0.5444 - val_loss: 0.5904 - val_accuracy: 0.5824
Epoch 6/10
45/45 [==============================] - 99s 2s/step - loss: 0.5829 - accuracy: 0.5611 - val_loss: 0.5768 - val_accuracy: 0.6593
Epoch 7/10
45/45 [==============================] - 98s 2s/step - loss: 0.5727 - accuracy: 0.6222 - val_loss: 0.5719 - val_accuracy: 0.5824
Epoch 8/10

## 1) Find Tuning


*   BS = 32
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001




##### Data

In [ ]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [01:58<00:00,  3.80it/s]


In [ ]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [ ]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

87924736/87910968 [==============================] - 0s 0us/step


311

In [ ]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [ ]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
INIT_LR = 1e-1
BS = 32
EPOCHS = 50

aug = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [ ]:
kf_IncepF = KFold(5, shuffle=True, random_state=40) 

oos_yF = []
oos_predF = []
fold = 0

for train, test in kf_IncepF.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF = data3_Incep[train]
    ytrain_IncepF = labels3_Incep[train]
    xtest_IncepF = data3_Incep[test]
    ytest_IncepF = labels3_Incep[test]

    M_IncepF = model3_IncepF.fit( x=aug.flow(xtrain_IncepF, ytrain_IncepF, batch_size=BS),
	                                            validation_data=(xtest_IncepF, ytest_IncepF),
	                                            steps_per_epoch=len(xtrain_IncepF) // BS,
	                                            epochs=15,  callbacks = [callback])
    
    pred_IncepF = model3_IncepF.predict(xtest_IncepF)
    
    oos_yF.append(ytest_IncepF)
    oos_predF.append(pred_IncepF)  
    print(classification_report(ytest_IncepF.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/15
11/11 [==============================] - 212s 18s/step - loss: 0.5453 - accuracy: 0.5701 - val_loss: 0.8733 - val_accuracy: 0.4396
Epoch 2/15
11/11 [==============================] - 188s 17s/step - loss: 0.3283 - accuracy: 0.7957 - val_loss: 0.8059 - val_accuracy: 0.4725
Epoch 3/15
11/11 [==============================] - 188s 17s/step - loss: 0.2771 - accuracy: 0.8476 - val_loss: 0.7430 - val_accuracy: 0.5604
Epoch 4/15
11/11 [==============================] - 188s 17s/step - loss: 0.1564 - accuracy: 0.9238 - val_loss: 0.7924 - val_accuracy: 0.5385
Epoch 5/15
11/11 [==============================] - 188s 17s/step - loss: 0.1670 - accuracy: 0.9207 - val_loss: 0.8088 - val_accuracy: 0.5714
Epoch 6/15
11/11 [==============================] - 188s 17s/step - loss: 0.1128 - accuracy: 0.9421 - val_loss: 0.8733 - val_accuracy: 0.5714
Epoch 7/15
11/11 [==============================] - 188s 18s/step - loss: 0.1176 - accuracy: 0.9299 - val_loss: 1.0824 - val_accuracy: 0.527

In [ ]:
oos_yF = np.concatenate(oos_yF)
oos_predF = np.concatenate(oos_predF)

In [ ]:
score = metrics.accuracy_score(oos_yF.argmax(axis=1),	        #ค่า acc
                                             oos_predF.argmax(axis=1) )
print(f"Fold score (accuracy): {score}")

print(classification_report(oos_yF.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

Fold score (accuracy): 0.9046563192904656
              precision    recall  f1-score   support

  Lungcancer       1.00      0.82      0.90       150
      Normal       0.99      0.90      0.94       151
Tuberculosis       0.78      0.99      0.88       150

    accuracy                           0.90       451
   macro avg       0.92      0.90      0.91       451
weighted avg       0.92      0.90      0.91       451



In [ ]:
#ลองเซฟโมเดล
fold = 2
if metrics.accuracy_score( ytest_IncepF.argmax(axis=1),  pred_IncepF.argmax(axis=1) )>0.7:
    name = (f' model3_IncepF{fold}')
    joblib.dump(model3_IncepF, name)

INFO:tensorflow:Assets written to: ram://a5d1d51e-3da4-454e-81fb-752e887bd981/assets


## 2) Find Tuning 

ลองรันอีกรอบ แบบที่เพิ่ม epoch 30

*   BS = 32
*   fix weight & bias in layer 0-100
*   learning_rate=0.0001


##### Data

In [12]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [03:09<00:00,  2.38it/s]


In [13]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [14]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

87924736/87910968 [==============================] - 2s 0us/step


311

In [ ]:
base_model3_Incep.summary()

In [16]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [17]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model3_IncepF2 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model3_IncepF2.summary()

In [19]:
INIT_LR = 1e-1
BS = 32
EPOCHS = 50

aug2 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.15,				  #เลื่อนลง-ขึ้น ซ้ายขวา
		height_shift_range=0.15,
		shear_range=0.15, 						#ขยับมุมภาพ ซ้าย และ/หรือ ขวา ไปด้านข้าง เช่น จะกลายเป็นสี่เหลี่ยมด้านขนาน 
		horizontal_flip=True,
		fill_mode='constant')

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [20]:
kf_IncepF2 = KFold(5, shuffle=True, random_state=40) 

oos_yF2 = []
oos_predF2 = []
fold = 0

for train, test in kf_IncepF2.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF2 = data3_Incep[train]
    ytrain_IncepF2 = labels3_Incep[train]
    xtest_IncepF2 = data3_Incep[test]
    ytest_IncepF2 = labels3_Incep[test]

    M_IncepF2 = model3_IncepF2.fit( x=aug2.flow(xtrain_IncepF2, ytrain_IncepF2, batch_size=BS),
	                                            validation_data=(xtest_IncepF2, ytest_IncepF2),
	                                            steps_per_epoch=len(xtrain_IncepF2) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF2 = model3_IncepF2.predict(xtest_IncepF2)

    oos_yF2.append(ytest_IncepF2)
    oos_predF2.append(pred_IncepF2) 

    score = metrics.accuracy_score(ytest_IncepF2.argmax(axis=1),	        #ค่า acc
                                               pred_IncepF2.argmax(axis=1) )
    print(f'Fold #{fold} score (accuracy): {score}')
    

#    if metrics.accuracy_score( ytest_IncepF2.argmax(axis=1),  pred_IncepF2.argmax(axis=1) )>0.9:
 #       name = (f' model3_IncepF2_Fold{fold}')
  #      joblib.dump(model3_IncepF2, name)

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
11/11 [==============================] - 179s 15s/step - loss: 0.4941 - accuracy: 0.6402 - val_loss: 1.2313 - val_accuracy: 0.3956
Epoch 2/30
11/11 [==============================] - 167s 15s/step - loss: 0.3458 - accuracy: 0.7927 - val_loss: 1.5341 - val_accuracy: 0.3846
Epoch 3/30
11/11 [==============================] - 166s 15s/step - loss: 0.2591 - accuracy: 0.8323 - val_loss: 1.5701 - val_accuracy: 0.3626
Epoch 4/30
11/11 [==============================] - 165s 15s/step - loss: 0.2039 - accuracy: 0.8872 - val_loss: 1.2643 - val_accuracy: 0.4286
Epoch 5/30
11/11 [==============================] - 163s 15s/step - loss: 0.1457 - accuracy: 0.9268 - val_loss: 1.3510 - val_accuracy: 0.4505
Epoch 6/30
11/11 [==============================] - 164s 15s/step - loss: 0.1662 - accuracy: 0.9055 - val_loss: 1.1395 - val_accuracy: 0.4615
Epoch 7/30
11/11 [==============================] - 164s 15s/step - loss: 0.0881 - accuracy: 0.9604 - val_loss: 0.8741 - val_accuracy: 0.593

AttributeError: ignored

In [21]:
oos_yF2 = np.concatenate(oos_yF2)
oos_predF2 = np.concatenate(oos_predF2)

In [22]:
print(classification_report(oos_yF2.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF2.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       1.00      0.49      0.66       150
      Normal       0.96      0.89      0.93       151
Tuberculosis       0.63      1.00      0.78       150

    accuracy                           0.80       451
   macro avg       0.87      0.80      0.79       451
weighted avg       0.87      0.80      0.79       451



## 3) Find Tuning 

*   activation="sigmoid"
*   fix weight & bias in layer 0-100




In [23]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = All_3class_dir +'/' + ImageName                   # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                       #ขนาดของ input
    train_img3_Incep.append(img)

data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

100%|██████████| 451/451 [00:05<00:00, 85.70it/s] 


In [24]:
labels3_Incep = np.array(data_train3[['NameType']] )        
labels3_Incep.shape = (len(labels3_Incep),) 

le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 

labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

##### Model

In [25]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model3_Incep.layers)

311

In [26]:
base_model3_Incep.trainable = True
for layer in base_model3_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False

In [27]:
#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)

prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model3_IncepF3 = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

model3_IncepF3.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

In [28]:
INIT_LR = 1e-1
BS = 16

aug3 = ImageDataGenerator(
		rotation_range=15,						#องศาการหมุน
		width_shift_range=0.1,
		height_shift_range=0.1,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

callback = tf.keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=3,
                mode="max" )

In [29]:
kf_IncepF3 = KFold(5, shuffle=True, random_state=40) 

oos_yF3 = []
oos_predF3 = []
fold = 0

for train, test in kf_IncepF3.split(data3_Incep):
    fold+=1
    print(f"Fold #{fold}")
        
    xtrain_IncepF3 = data3_Incep[train]
    ytrain_IncepF3 = labels3_Incep[train]
    xtest_IncepF3 = data3_Incep[test]
    ytest_IncepF3 = labels3_Incep[test]

    M_IncepF3 = model3_IncepF3.fit( x=aug3.flow(xtrain_IncepF3, ytrain_IncepF3, batch_size=BS),
	                                            validation_data=(xtest_IncepF3, ytest_IncepF3),
	                                            steps_per_epoch=len(xtrain_IncepF3) // BS,
	                                            epochs=30,  callbacks = [callback])
    
    pred_IncepF3 = model3_IncepF3.predict(xtest_IncepF3)
    
    oos_yF3.append(ytest_IncepF3)
    oos_predF3.append(pred_IncepF3)  
    print(classification_report(ytest_IncepF3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_IncepF3.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

#   score = metrics.accuracy_score(ytest_IncepF.argmax(axis=1),	        #ค่า acc
#                                               pred_IncepF.argmax(axis=1)
#    print(f"Fold score (accuracy): {score}")

#    print(confusion_matrix(ytest_IncepF.argmax(axis=1),	                       #แสดง confusion_matrix
#                        pred3_IncepF.argmax(axis=1) ) )

Fold #1
Epoch 1/30
22/22 [==============================] - 185s 8s/step - loss: 0.7111 - accuracy: 0.3634 - val_loss: 11.0639 - val_accuracy: 0.3626
Epoch 2/30
22/22 [==============================] - 173s 8s/step - loss: 0.5690 - accuracy: 0.5523 - val_loss: 6.6138 - val_accuracy: 0.4286
Epoch 3/30
22/22 [==============================] - 173s 8s/step - loss: 0.5049 - accuracy: 0.6047 - val_loss: 5.6047 - val_accuracy: 0.3736
Epoch 4/30
22/22 [==============================] - 172s 8s/step - loss: 0.4228 - accuracy: 0.7064 - val_loss: 4.8709 - val_accuracy: 0.3846
Epoch 5/30
22/22 [==============================] - 175s 8s/step - loss: 0.3286 - accuracy: 0.7500 - val_loss: 1.4466 - val_accuracy: 0.6484
Epoch 6/30
22/22 [==============================] - 172s 8s/step - loss: 0.3368 - accuracy: 0.7616 - val_loss: 1.5212 - val_accuracy: 0.6593
Epoch 7/30
22/22 [==============================] - 171s 8s/step - loss: 0.2781 - accuracy: 0.7878 - val_loss: 45.6544 - val_accuracy: 0.3407
Epo

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  Lungcancer       0.00      0.00      0.00        33
      Normal       0.72      0.50      0.59        26
Tuberculosis       0.39      0.90      0.54        31

    accuracy                           0.46        90
   macro avg       0.37      0.47      0.38        90
weighted avg       0.34      0.46      0.36        90

Fold #3
Epoch 1/30
22/22 [==============================] - 174s 8s/step - loss: 0.1925 - accuracy: 0.8870 - val_loss: 6.6523 - val_accuracy: 0.5222
Epoch 2/30
22/22 [==============================] - 174s 8s/step - loss: 0.1293 - accuracy: 0.9246 - val_loss: 3.3969 - val_accuracy: 0.6556
Epoch 3/30
22/22 [==============================] - 171s 8s/step - loss: 0.1289 - accuracy: 0.9246 - val_loss: 1.4979 - val_accuracy: 0.7444
Epoch 4/30
22/22 [==============================] - 172s 8s/step - loss: 0.1009 - accuracy: 0.9391 - val_loss: 2.5787 - val_accuracy: 0.6111
Epoch 5/30
22/22 [=============================

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  Lungcancer       0.00      0.00      0.00        29
      Normal       0.97      0.97      0.97        33
Tuberculosis       0.47      0.96      0.64        28

    accuracy                           0.66        90
   macro avg       0.48      0.64      0.53        90
weighted avg       0.50      0.66      0.55        90

Fold #4
Epoch 1/30
22/22 [==============================] - 172s 8s/step - loss: 0.1583 - accuracy: 0.9217 - val_loss: 3.6734 - val_accuracy: 0.5889
Epoch 2/30
22/22 [==============================] - 172s 8s/step - loss: 0.0844 - accuracy: 0.9565 - val_loss: 0.1338 - val_accuracy: 0.9444
Epoch 3/30
22/22 [==============================] - 173s 8s/step - loss: 0.1214 - accuracy: 0.9304 - val_loss: 0.6482 - val_accuracy: 0.8444
Epoch 4/30
22/22 [==============================] - 173s 8s/step - loss: 0.0886 - accuracy: 0.9507 - val_loss: 0.6135 - val_accuracy: 0.8333
Epoch 5/30
22/22 [=============================

In [30]:
oos_yF3 = np.concatenate(oos_yF3)
oos_predF3 = np.concatenate(oos_predF3)

In [31]:
print(classification_report(oos_yF3.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        oos_predF3.argmax(axis=1),  
                                        target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       1.00      0.23      0.37       150
      Normal       0.76      0.88      0.81       151
Tuberculosis       0.52      0.84      0.64       150

    accuracy                           0.65       451
   macro avg       0.76      0.65      0.61       451
weighted avg       0.76      0.65      0.61       451



In [ ]:
#ลองเซฟโมเดล
fold = 2
if metrics.accuracy_score( ytest_IncepF.argmax(axis=1),  pred_IncepF.argmax(axis=1) )>0.7:
    name = (f' model3_IncepF{fold}')
    joblib.dump(model3_IncepF, name)

INFO:tensorflow:Assets written to: ram://a5d1d51e-3da4-454e-81fb-752e887bd981/assets
